# Pixel Analyzing Tool (PAT) Jupyter Notebook 

A tool for computing statistical values for images using CIAO DMSTAT tool.

### This Notebook is an example tutorial.

#### Import Libraries

In [1]:
import numpy as np
from astropy.table import Table
from astropy.io import fits
import pandas as pd
import os
from ciao_contrib.runtool import dmstat

print("Libraries Successfully Imported!")

Libraries Successfully Imported!


Let's read in the fits file.

In [2]:
hdu_list = fits.open('y3a2-6.4.22+2_peak_fixed_maybe_final_merged.fits')

We'll create a FITS table object using the fits file.

In [3]:
Cat = Table.read('y3a2-6.4.22+2_peak_fixed_maybe_final_merged.fits')

Now let's go ahead and create a Pandas DataFrame using the FITS object. 
We'll call it `df_catalog` and display the first 10 rows.

In [4]:
df_catalog = Cat.to_pandas()

print(f"{df_catalog.shape[0]} catalogs and {df_catalog.shape[1]} features.")
df_catalog.head(10)

1092 catalogs and 110 features.


,Name,MEM_MATCH_ID,Analyzed,Detected,redMaPPer_ra,redMaPPer_dec,Redshift,Obsids,lambda,lambda_err_low,...,overlap bkgd,notes,Group 1 Check,redmapper wrong,right RA,right DEC,right in top 5,right not member,ambiguous,Chandra Notes
0,b'catalogue_1',2,True,True,83.231902,-37.026689,0.283338,b'15112',199.432,5.87942,...,0,b'N/A',b'yes',0,9999.00000,9999.000000,0,0,0,b'N/A'
1,b'catalogue_2',3,True,True,347.092550,-2.192137,0.292506,"b'4962, 9372'",163.583,4.29459,...,0,"b'not clear why no temp, prob too big in S7'",b'yes',0,9999.00000,9999.000000,0,0,0,b'N/A'
2,b'catalogue_3',4,True,True,79.155685,-54.500459,0.298755,"b'9331, 15099'",207.243,7.18140,...,0,b'N/A',b'yes',0,9999.00000,9999.000000,0,0,0,"b'gas a bit offset from galaxies, but not too ..."
3,b'catalogue_5',6,True,True,62.795693,-48.327679,0.412681,"b'13396, 16355, 17536'",178.045,5.02932,...,0,b'only r2500',b'yes',0,9999.00000,9999.000000,0,0,1,"b'red picks one substr and peak the other, cen..."
4,b'catalogue_6',7,True,True,351.298843,-41.203703,0.357925,"b'13405, 19774'",176.001,4.71629,...,0,b'N/A',b'yes',0,9999.00000,9999.000000,0,0,0,b'N/A'
5,b'catalogue_7',8,True,True,41.353370,-53.029274,0.298389,"b'12260, 16127, 16282, 16524, 16525, 16526'",150.571,4.14139,...,0,b'N/A',b'yes',0,9999.00000,9999.000000,0,0,0,"b'merger, X-ray and red agree but another clum..."
6,b'catalogue_10',11,True,True,64.038120,-24.067486,0.390756,"b'10446, 16236, 16237, 16304, 16523, 17313'",165.811,5.31168,...,0,b'N/A',b'yes',0,9999.00000,9999.000000,0,0,1,b'peak and red agree but merger with multiple ...
7,b'catalogue_13',15,True,True,342.237897,-44.502982,0.349226,"b'4966, 18611, 18818'",160.700,6.59675,...,0,"b'mispercolated, X-ray cluster found as lambda...",b'yes',1,342.18318,-44.530803,4,0,0,b'mispercolated'
8,b'catalogue_14',16,True,True,39.969643,-1.571880,0.372599,"b'515, 7715'",165.521,4.71158,...,0,b'N/A',b'yes',0,9999.00000,9999.000000,0,0,1,b'other central in merger is second most likel...
9,b'catalogue_17',19,True,True,10.209377,-44.131350,0.360712,b'13395',142.913,3.74835,...,0,b'N/A',b'yes',0,9999.00000,9999.000000,0,0,0,b'N/A'


#### Pre-process

We only need two features from `df_catalog`, the `Name` and `Obsids`. Lets go ahead and isolate those two features in new Dataframe called `df_obsid`

In [5]:
df_obsid = df_catalog.loc[:,['Name', 'Obsids']]
df_obsid.head()

,Name,Obsids
0,b'catalogue_1',b'15112'
1,b'catalogue_2',"b'4962, 9372'"
2,b'catalogue_3',"b'9331, 15099'"
3,b'catalogue_5',"b'13396, 16355, 17536'"
4,b'catalogue_6',"b'13405, 19774'"


Note the values in each column contain a wrapper text b' '.

These are known as byte-strings and we'll decode then as follows.  

In [6]:
df_obsid['Name'] = df_obsid['Name'].str.decode('ASCII') 
df_obsid['Obsids'] = df_obsid['Obsids'].str.decode('ASCII') 

Check the dataframe.

In [7]:
print(f"{df_obsid.shape[0]} catalogs and {df_obsid.shape[1]} features.")
df_obsid.head()

1092 catalogs and 2 features.


,Name,Obsids
0,catalogue_1,15112
1,catalogue_2,"4962, 9372"
2,catalogue_3,"9331, 15099"
3,catalogue_5,"13396, 16355, 17536"
4,catalogue_6,"13405, 19774"


Check for any NA/NaN/None values.

In [8]:
df_obsid.isna().count()

Name      1092
Obsids    1092
dtype: int64

The `Obsids` feature contains the numbers we are interested but the entire value is of dtype string, not a list.

Let's create a list of each of the feature values and create a new feature that will replace the existing `Obsids`.

In [9]:
# start by creating an empty list to hold values.
list_of = []
for obs in df_obsid['Obsids']:
    obs = obs.split(",")
    list_of.append(obs)
    
#print(list_of) # We have the option to display the list contents.

# Add in a new column/feature.
df_obsid['New_col'] = list_of

Check the updated `df_obsid`

In [10]:
print(f"{df_obsid.shape[0]} catalogs and {df_obsid.shape[1]} features.")
df_obsid.head()

1092 catalogs and 3 features.


,Name,Obsids,New_col
0,catalogue_1,15112,[15112]
1,catalogue_2,"4962, 9372","[4962, 9372]"
2,catalogue_3,"9331, 15099","[9331, 15099]"
3,catalogue_5,"13396, 16355, 17536","[13396, 16355, 17536]"
4,catalogue_6,"13405, 19774","[13405, 19774]"


Now we have a new feature with the obsids as lists.

* Let's drop 'Obsids'
* Rename 'New_col' with 'Obsid'.

In [11]:
df_obsid.drop('Obsids', axis=1, inplace=True)
df_obsid.rename(columns={'New_col':'Obsid'}, inplace=True)

df_obsid.head()

,Name,Obsid
0,catalogue_1,[15112]
1,catalogue_2,"[4962, 9372]"
2,catalogue_3,"[9331, 15099]"
3,catalogue_5,"[13396, 16355, 17536]"
4,catalogue_6,"[13405, 19774]"


Check if we actually have lists under `Obsid`

In [12]:
#for i, l in enumerate(df_obsid["Obsid"]):
#    print(f"list {i} is {type(l)}")

We need to extract the catalog number with its corresponding obsid number. 

To do that we use the zip() function. 

In [13]:
#for cat_, obs_ in zip(df_obsid['Name'], df_obsid['Obsid']):
#   for element in obs_[:]:
#       pass
#   #print("This is {} with obsid: {}".format(cat, obs))

#### DMSTAT

Let's create a function that runs dmstat on all the catalogs and outputs a DataFrame with DMSTAT Output values. 

In [14]:
def r500_source(dataframe):
    """ Used to calculate statistics on r500 source region files 
        and outputs a Pandas DataFrame created from CIAO DMSTAT 
        output data.
    """
    
    df = dataframe
    
    #empty list
    cat_obs_list = []
    min_list = []
    max_list = []
    mean_list = []
    sigma_list = []
    good_list = []
    null_list = []
    median_list = []
    
    print("Creating DataFrame")
    
    for cat, obs in zip(df['Name'], df['Obsid']):
        for e in obs[:]:
            
            print(".", sep=' ', end='', flush=True)
            
            if os.path.exists("/data1/devon/y3a2/current/observations/{}/I/flux_band1_thresh.expmap".format(e)
                             ) == True:
                if os.path.exists("/data1/devon/y3a2/current/clusters/{}/region_{}_r500_source.reg".format(cat,e)
                                 ) == True:
                    
                    # set v=0 to disbale dmstat display
                    Stats=dmstat("/data1/devon/y3a2/current/observations/{}/I/flux_band1_thresh.expmap["\
                    "sky=region(/data1/devon/y3a2/current/clusters/{}/region_{}_r500_source.reg)]".format(e,cat,e),
                                 centroid=False,
                                 median=True,
                                 sigma=True
                                )
                    # Output parameters
                    median = dmstat.out_median
                    minimum = dmstat.out_min
                    maximum = dmstat.out_max
                    mean = dmstat.out_mean 
                    sigma = dmstat.out_sigma
                    columns = dmstat.out_columns
                    good = dmstat.out_good
                    null = dmstat.out_null
                    
                    #print("{} with Obs-id: {}".format(cat, e)) # add Stats to argument to print default screen output                    
                    #print("{}".format(columns))                # toolname.parmname format                     
                    #print("Median = {}".format(median))                     
                    #print("Minimum = {}".format(minimum))                    
                    #print("Maximum = {}".format(maximum))                    
                    #print("Mean = {}".format(mean))                    
                    #print("Sigma = {}".format(sigma))                    
                    #print("Number of Good Values = {}".format(good))                    
                    #print("Number of Null Values = {}".format(null))                    
                    #print(end="\n")
                    
                    min_list.append(minimum)
                    max_list.append(maximum)
                    mean_list.append(mean)
                    sigma_list.append(sigma)
                    good_list.append(good)
                    null_list.append(null)
                    median_list.append(median)
                    cat_obs_list.append((cat,e))
                
            else:
                pass
            
    # Create empty DataFrame
    df_r500_src = pd.DataFrame()
    
    df_r500_src["Catalog"] = pd.Series(cat_obs_list)
    df_r500_src["Minimum"] = pd.Series(min_list)
    df_r500_src["Maximum"] = pd.Series(max_list)
    df_r500_src["Mean"] = pd.Series(mean_list)
    df_r500_src["Sigma"] = pd.Series(sigma_list)
    df_r500_src["Good"] = pd.Series(good_list)
    df_r500_src["Null"] = pd.Series(null_list)
    df_r500_src["Median"] = pd.Series(median_list)
    
    print("DONE!")
    return df_r500_src

Let's call `r500_source()` and give it the `df_obsid`.

In [15]:
df_r500_src = r500_source(df_obsid)

print(f"{df_r500_src.shape[0]} catalogs analyzed. {df_r500_src.shape[1]} Features.")
df_r500_src.head(10) 

Creating DataFrame
.....................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

,Catalog,Minimum,Maximum,Mean,Sigma,Good,Null,Median
0,"(catalogue_1, 15112)",0,6096060.5,4956312.7507,1026116.0008,1602278,459818,5212436.5
1,"(catalogue_3, 9331)",0,2586893.5,2178645.4199,418113.96817,1376803,412103,2284914.75
2,"(catalogue_5, 13396)",323776.40625,2051581.375,1877169.3389,182020.85639,508650,140986,1918851.3125
3,"(catalogue_6, 13405)",81689.773438,2331608,1985969.771,339724.97557,1035859,288942,2071867.25
4,"(catalogue_7, 12260)",0,5106971.5,4185653.6955,861707.66874,1271653,369308,4386643
5,"(catalogue_10, 10446)",161333.625,4295240.5,3714437.1135,701896.99261,863497,239003,3958583.25
6,"(catalogue_13, 4966)",290992,7794266,6688937.6221,1194683.0259,1330319,383162,7040313
7,"(catalogue_17, 13395)",293169.8125,1997733,1741614.517,289505.36092,732623,202466,1820484.125
8,"(catalogue_20, 15097)",0,4479037,3634700.407,723164.63477,2372246,734160,3791811.125
9,"(catalogue_21, 3248)",0,2808435.75,2344410.91,499657.74744,1291604,381538,2475580


Great! Save the DataFrame to a CSV file.

In [17]:
os.mkdir("/home/jose/PAT_Data")

df_r500_src.to_csv(r"/home/jose/PAT_Data/r500_src_df_2.csv")

That's it! 